In [1]:
import glob

import polars as pl

In [2]:
DATA = './data/nyc-taxi/*.parquet'
files = list(glob.glob(DATA))

In [3]:
%%time

# this method avoids a PanicException: this operation is not implemented/valid for this dtype: Null
df = pl.concat(
    [
        pl.scan_parquet(file).drop('airport_fee').lazy()
        for file in files
    ]
)

CPU times: user 7.22 ms, sys: 9.34 ms, total: 16.6 ms
Wall time: 27.8 ms


In [4]:
%%time

(
    df.with_columns(
        is_congested=pl.col('congestion_surcharge').fill_null(0) > 0
    )
    .groupby('DOLocationID')
    .agg(
        mean_fare=pl.mean('fare_amount'),
        count=pl.count(),
        congestion=pl.mean('is_congested')
    )
    .sort('count', descending=True)
    .collect()
)

CPU times: user 14.7 s, sys: 23.8 s, total: 38.5 s
Wall time: 19.1 s


DOLocationID,mean_fare,count,congestion
i64,f64,u32,f64
236,9.535313,7540155,0.930053
237,9.245067,7102646,0.940954
161,10.944027,6428273,0.935369
170,10.735888,5376756,0.932148
162,11.099968,4948884,0.934476
230,13.361038,4882887,0.933714
142,10.222429,4793638,0.935075
48,11.880889,4742027,0.931868
239,10.4412,4680690,0.935553


## Kernel Dies, Below:

In [ ]:
df.collect().shape

In [ ]:
df.collect().estimated_size('gb')

In [ ]:
df.collect().head()